# **ECON 585 Final Project**
### By Peterson, Seth and Vorkink, Eric

## **SECTION 0:** Setting Up Development Environment

In [2]:
# The FamaData class was created by Seth Peterson for purpose of the 588 Project
from fama_data.FamaData import FamaData

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

## **SECTION 1:** Importing and Gathering Data